---
## Комментарий от наставника

Привет! Спасибо, что прислал задание:) Поздравляю с приближением к концу первого модуля. Мои комментарии, как обычно, обозначены пометкой **Комментарий от наставника**. Пострайся учесть эти комментарии.

---

---
## Комментарий от наставника


Стоит делать краткое вступление в работу, описание того, что надо делать. В работе необходимо приводить краткий план того, что надо сделать (если этот план имеется), а также информацию о входных данных: какие столбцы есть в таблице, их названия и какую информацию они несут. Также название работы лучше отображать в отдельной ячейке и делать крупный шрифт (заголовок). Так работа выглядит презентабельно.

---

# Библиотеки для работы

In [1]:
# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# будем отображать графики прямо в jupyter'e
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 12, 5
import pandas as pd
import numpy as np

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go

init_notebook_mode(connected=True)

from scipy import stats as st

# Ознакомление с данными

In [2]:
games = pd.read_csv('/datasets/games.csv').copy()

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/games.csv'

In [ ]:
games.info()

In [ ]:
games.columns = map(str.lower, games.columns)

In [ ]:
games

### Датафрейм довольно большой, много пропусков. Особенно в полях с рейтингом (возрастным, пользователей, критиков). Есть еще ошибка в формате: дата с форматом float64, оценка пользователей - текст. 

---
## Комментарий от наставника

Подумай также о возможных причинах появления пропусков, а также о том, являются ли они случайными или нет. Можно высказать варианты их обработки. Также важно делать выводы после каждого пункта анализа. 

Первый взгляд на таблицу выполнен.

---

# Подготовка данных

In [ ]:
games['year_of_release'] = pd.to_datetime(games['year_of_release'] , format = "%Y") #сменим тип данных 

In [ ]:
games.query('user_score == "tbd"') 

### TBD - отсутствие рейтинга, следственно его лучше заменить на NaN, так как по сути он указывает на отсутствие рейтинга. 

---
## Комментарий от наставника

Верно, по своей сути tbd и является Nan. 

---

In [ ]:
games = games.replace('tbd', np.nan)

In [ ]:
games['user_score'] = games['user_score'].astype('float64')

In [ ]:
games.isna().sum() #посмотрим пропуски 

In [ ]:
games = games.dropna(subset = ['name', 'year_of_release' ,'genre'])

### Пропуски можно безболезненно выкинуть из столбцов 'name', 'year_of_release' ,'genre'. В остальных столбцах лучше оставить пропуски, не меняя на 0. 

## Добавим новый столбец с общими продажами по миру 

In [ ]:
games['world_sales'] = games['na_sales']+games['eu_sales']+games['jp_sales']+games['other_sales']

In [ ]:
games['year_of_release'].hist()
plt.show();

---
## Комментарий от наставника

Да, лучше работать с меньшим количеством данных хорошего качества. Можно использовать индикаторные значения для заполнения пропущенных значений. Так мы просто пометим места пропусков. Главное помнить об этом и не использовать их в анализе.

---

# Анализ современности 

## Для анализа отберем современные данные, чтобы лучше понимать ситуацию на рынке на сегодняшний день

In [ ]:
modern_games = games.query('year_of_release > 2010')
modern_games

---
## Комментарий от наставника

Выбран правильный актуальный год. 

---

### Посмотрим платформы по популярности

In [ ]:
modern_games['platform'].value_counts().plot(kind='bar');
plt.title('Игровые платформы')
plt.show();

### Больше всего игр выходило для приставок PS, так же много игры у приставок от майкрософт. 

### Объем продаж по платформам

In [ ]:
modern_games.pivot_table(index = 'platform', 
                 values = 'world_sales', 
                 aggfunc ='sum').sort_values(by='world_sales',
                                             ascending = False).head(10).plot(kind='bar');
plt.title('Продажи по платформам')
plt.ylabel('Млн. $')
plt.xlabel('Платформы')
plt.show();

### Ситуация похожая с количеством игр: чем больше игр выпускается на платформе, тем больше суммарные продажи. 

## Посмотрим на продажи поподробнее

In [ ]:
sales = modern_games.pivot_table(index = ['platform', 'year_of_release'], 
                                 values = 'world_sales', 
                                 aggfunc = 'sum')
sales = sales.reset_index()

In [ ]:
sns.boxplot(x="platform", y="world_sales", data=sales, palette='deep')
plt.show();

In [ ]:
sns.lineplot(x="year_of_release", y="world_sales", hue = 'platform', data=sales, palette="deep");
plt.title('Продажи по платформам')
plt.ylabel('Млн. $')
plt.xlabel('Платформы')
plt.show();

### Динамика у всех платформ всегда одна и та же: сразу после выхода платформы разработчики делают большое количество игр, заполняя нишу, продажи быстро и уверенно растут, но продажи цикличны, после сильного роста они замедляются и начиают падать. Это связано с устареванием платформы, разработчики не тратят много сил на разработку больших проектов, сосредотачиваются на обновлениях и DLC. Как только выходит следующее поколение в платформе, цикл повторяется.

---
## Комментарий от наставника

Какова средняя продолжительность существования платформы?  

---

## Взаимосвязь данных 

In [ ]:
sns.pairplot(modern_games, vars=["critic_score",
                                 "user_score", 
                                 'world_sales'], markers = "+")
plt.show();

In [ ]:
modern_games[["critic_score",
                                 "user_score", 
                                 'world_sales']].corr()

### Видно весьма слабую взаимосвязь между оценками и продажами по миру, но самая сильная связь между оценками критиков и пользователей. То ли пользователи вторят критикам, то ли критики весьма близки к народу.

---
## Комментарий от наставника

Результат верный. Здорово, что высказано предположение о таких результатах. Стоит рассмотреть некоторые платформы в отдельности. 
Возможно, ситуация у них разнится. 

---

# Жанры 

In [ ]:
# создаем Box trace для каждого жанра из наших данных
data = []
for genre in modern_games.genre.unique():
    data.append(
        go.Box(y=modern_games[(modern_games.genre==genre) & (modern_games.world_sales <2.5)].world_sales, name=genre)
    )

# визуализируем данные
iplot(data, show_link = False)


### Уберем сильные выбросы (что больше 2,5 миллионов) и посмотрим на продажи по жанрам. Самые большие продажи в шутерах, платформерах и и спортивных играх, следом идут гонки экшн и файтинги. Сильно мало продаж у приключенческих игр и пазлов. 

---
## Комментарий от наставника

Правильно, но не стоит забывать, что производство игр в жанрах Action или Shooter обходится сильно дороже, чем производство Puzzle-игр. 

---

# Продажи по регионам

In [ ]:
region_sales = modern_games.pivot_table(index=['platform','genre'], values = ['na_sales',
                                                     'eu_sales',
                                                     'jp_sales', 
                                                     'other_sales', 'world_sales'], 
                        aggfunc = 'sum').reset_index()
region_sales

## Посмотрим на пользователей из Японии. 

In [ ]:
j_users = region_sales[['platform', 'genre','jp_sales']] .sort_values(by = 'jp_sales', ascending = False).head()
j_users

In [ ]:
sns.catplot(
    x="platform", y="jp_sales", hue="genre", data=j_users, palette="deep", kind = 'bar')
plt.show();

### Больше всего продаж в японии на платформах 3DS и PS3, причем чаще всего играют в жанры RPG, экшн и симуляторы. Посмотрим на топ-5 жанров. 

In [ ]:
j_users_genre = region_sales.pivot_table(index='genre', values ='jp_sales', 
                                         aggfunc = 'sum') .sort_values(by = 'jp_sales',
                                                                       ascending = False).head().reset_index()

In [ ]:
sns.catplot(
    x="genre", y="jp_sales",  data=j_users_genre, palette="deep", kind = 'bar')
plt.show();

In [ ]:
rating_pivot_jp  = modern_games.pivot_table(index = 'rating',
                                            values = 'jp_sales', aggfunc = 'sum').sort_values(by = 'jp_sales',
                                                                                              ascending = False).head().reset_index()
sns.catplot(
    x="rating", y="jp_sales",  data=rating_pivot_jp, palette="deep", kind = 'bar')
plt.show();

### В Японии больше всего играют в знаменитые японские RPG и экшн игры, сразу за ними игры смешенного жанра и платформеры в файтингами - стандартные жанры для японского рынка.  В Японии видно так же, что больше всего продаж сосредоточено в играх с рейтингом E и M. 

## Пользователи северной Америки

In [ ]:
na_users = region_sales[['platform','genre','na_sales']] .sort_values(by = 'na_sales', ascending = False).head()
na_users

In [ ]:
sns.catplot(
    x="platform", y="na_sales", hue="genre", data=na_users, palette="deep", kind = 'bar')
plt.show();

### Больше всего продаж в Америке приходится на платформах от майкрософт и PS3, причем чаще всего играют в жанры экшн и шутеры. Американцы поддерживают внутренний своего производителя, а японцы своего - все логично. Так же в глаза бросается разница в объемах - рынок Америки сильно больше.  Посмотрим на топ-5 жанров. 

In [ ]:
na_users_genre = region_sales.pivot_table(index='genre', values ='na_sales',
                                          aggfunc = 'sum') .sort_values(by = 'na_sales',
                                                                        ascending = False).head().reset_index()
na_users_genre

In [ ]:
sns.catplot(
    x="genre", y="na_sales",  data=na_users_genre, palette="deep", kind = 'bar')
plt.show();

In [ ]:
rating_pivot_na  = modern_games.pivot_table(index = 'rating',
                                            values = 'na_sales', aggfunc = 'sum').sort_values(by='na_sales', 
                                                                                              ascending = False).head().reset_index()
sns.catplot(
    x="rating", y="na_sales",  data=rating_pivot_na, palette="deep", kind = 'bar')
plt.show();

### Самые популярные - экшн и шутеры, так же американцы любят очень спортивные игры. Меняются данные по рейтингу - в лидеры вышли игры с рейтингом M, расчитанные на аудиторию от 17 лет.  

## Пользователи Еропы 

In [ ]:
eu_users = region_sales[['platform','genre','eu_sales']] .sort_values(by = 'eu_sales', ascending = False).head()
eu_users

In [ ]:
sns.catplot(
    x="platform", y="eu_sales",  data=eu_users, palette="deep", kind = 'bar')
plt.show();

### По платформам европейцы предпочитают PS, объем рынка побольше чем в Японии, поменьше чем в Америке. 

In [ ]:
eu_users_genre = region_sales.pivot_table(index='genre',
                                          values ='eu_sales', 
                                          aggfunc = 'sum') .sort_values(by = 'eu_sales',
                                                                        ascending = False).head().reset_index()

In [ ]:
sns.catplot(
    x="genre", y="eu_sales",  data=eu_users_genre, palette="deep", kind = 'bar')
plt.show();

### Жанры не сильно отличаются от Америки, но добавились гонки - европейцы традиционно их любят. 

In [ ]:
rating_pivot_eu  = modern_games.pivot_table(index = 'rating',
                                         values = 'eu_sales', 
                                         aggfunc = 'sum').sort_values(by='eu_sales', ascending = False).head().reset_index()

In [ ]:
sns.catplot(
    x="rating", y="eu_sales",  data=rating_pivot_eu, palette="deep", kind = 'bar')
plt.show();

### Распределение выручки по рейтингу тоже не сильно отличается от Америки, разве что чуть больше продажи по играм для подростков - рейтинг T. 

---
## Комментарий от наставника

Портрет типичного пользователя каждого из регионов получен. Стоит визуализировать все полученные таблицы. Здорово, что отмечены индивидуальные особенности каждого региона. 

---

# Проверка гипотез

### Сформируем две выборки по платформам с проставленными рейтингами пользователей. 

In [ ]:
XOne  = modern_games.query('platform == "XOne"').dropna()

In [ ]:
PC  = modern_games.query('platform == "PC"').dropna()

H0: Средние пользовательские рейтинги на плафтформах одинаковые. 

H1: Средние пользовательские рейтинги на плафтформах различаются. 

In [ ]:
mean_x_score = XOne['user_score'].mean()

alpha = 0.05

results = st.ttest_1samp(PC['user_score'], mean_x_score)

print('p-значение: ', results.pvalue)
if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

### Не удалось опровергнуть нулевую гипотез. Фактически средняя оценка почти не различается, небольшое различие скорее обуславливается случайным фактором. 

In [ ]:
mean_x_score

In [ ]:
PC['user_score'].mean()

### Сформируем две выборки по жанрам.

In [ ]:
act  = modern_games.query('genre == "Action"').dropna()
spt  = modern_games.query('genre == "Sports"').dropna()


In [ ]:
act_mean = act['user_score'].mean()

alpha = 0.05

results = st.ttest_1samp(spt['user_score'], act_mean)

print('p-значение: ', results.pvalue)
if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

In [ ]:
act_mean

In [ ]:
spt['user_score'].mean()

### Получилось опровергнуть нулевую гипотезу - рейтинг между жанрами экшн и спорт - отличаются. И это отличие можно признать статистически значимым. 

---
## Комментарий от наставника

Гипотезы сформулированы и проверены верно.

---

# Вывод: 

Мы видем, что за все время игровая индустрия явно набирает обороты, с 80-х она приросла в общем объеме многократно, но с 2012 года объем немного упал. Но и рынок поменялся, все меньше появляется новых платформ, все больше рынок держат устоявшиеся игроки, студии стараются выпускать игры для нового поколения платформ. Старое постепенно теряет в объеме продаж и количестве игр. Важными данными задающими тренд будем считать данные за текущее десятилетие, чтобы можно было захватить хотя бы два поколения основных платформ. 

Самые популярные платформы - PS  и приставки Xbox. Видно, что свежее поколение (PS4 и XOne) только набираются обороты, так же впечатляет объем приставок от Nintendo, но они традиционно сосредотачиваются на локальном рынке - Японии. 

Можно отметить, что объем рынка сейчас падает, но нет возможности установить с чем это может быть связано. Можно предположить, что эта доля перешла во free to play игры и скорее всего их деньги и сосредоточены там, так же не стоит забывать, что за последние 10 лет сильно поменялось соотношение онлайн/офлайн игр, онлайн начинать забирать значительную часть рынка, но продажи там считают редко - многие из этих игры условно бесплатные. 

Так же можно отметить сильную солидарность между критиками и пользователями, скорее всего у них одинаковые критерии оценки игр, да и само общество геймеров довольно серьезно оценивает все работы студий. 

Больше всего люди предпочитают играть в шутеры и платформеры, так же люди любят спортивные игры и экшн. 

Что касается регионов, то основными являются: Япония, северная Америка, Европа. Видно, что Америка и Япония поддерживают свои платформы (Xbox и PS соответсвенно), но при это PS предпочитают европейцы (хороший у них вкус, я вот сам только с PS с 2000 года), интересно, что представители западной культуры больше любят активные и агрессивные игры (экшн, шутеры), а японцы склонны с созерцательным жанрам (RPG). 


Гипотезы: 

Нельзя сказать, что рейтинг на PC и XBOX  различается, что логично - обе платформы имеют практически одинаковый игровой состав. 

Но вот рейтинги между жанром action и sports  - различаются. Думаю, это обуславливается тем, что action лучше продается, следственно, игры с большим бюджетом стараются сделать как можно качественнее. В то время как хороших спортивных симуляторов можно пересчитать по пальцам. 



---
## Комментарий от наставника

Выводы описаны грамотно. Приведены ответы на главные вопросы проекта. В выводах можно приводить полученные ранее значения, правильно. Также можно расписать все, что было сделано в работе.  

---

## Комментарий от наставника

Если тебе нравится тема визуализации, то можешь изучить другие методы библиотеки seaborn. Она позволяет строить довольно презентабельные графики.

Ты проделал большую работу, молодец! Критических замечаний нет. Работа выполнена хорошо. Продолжай в том же духе :)

-----